<a href="https://colab.research.google.com/github/indusconn/covid19-worldometer-scraper/blob/main/Covid19StaticsScraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

headers = {'User-Agent': 'Mozilla/5.0'}
#countries = []

def getCountriesList():
    print('*****************Getting Countries')
    url = "https://www.worldometers.info/coronavirus/#countries"
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")

    countriesTempo = []
    for a in soup.find_all('a', class_="mt_a", href=True):
        #print("Found the URL:", a['href'])
        countriesTempo.append(a['href'])

    global countries
    countries = []
    for word in countriesTempo:
        if word not in countries:
            countries.append(word)

    #print(countries)
    print(len(countries))


def searchSoup(text, data):
    loopCntr = 0
    textFound = False
    for dataCheck in data:
        if (dataCheck.string.find(text)) != -1:
            #print(dataCheck.string)
            textFound = True
            break
        loopCntr += 1
    if textFound:
        #print(loopCntr)
        return loopCntr
    else:
        print('-1')
        return -1


def getCountryStatics(url):
    request = requests.get(url, headers=headers)
    soup = BeautifulSoup(request.text, 'html.parser')

    #print('*****************soup for:')
    print(url)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    data = soup.find_all(
        'script', type='text/javascript', text=re.compile("Highcharts.chart"))
    #print(data[0])

    #print('*****************Search patterns')
    patternData = re.compile('data: \[(.*?)\]')
    patternCategories = re.compile('categories: \[(.*?)\]')

    #print('*****************Date Extraction')
    index = searchSoup('categories: [', data)
    substring = re.search(patternCategories, data[index].string)
    #print(substring[1].replace('\"',''))
    dayDate = pd.Series(
        substring[1].replace('\"', '').split(','), dtype="string", name='Date')
    #print(dayDate)

    #print('*****************Total Cases')
    index = searchSoup('Total Cases', data)
    #print(index)
    substring = re.search(patternData, data[index].string)
    #print(substring[1])
    totalCases = pd.Series(
        substring[1].split(','), name='Total Cases').astype(int)
    #print(totalCases)

    #print('*****************Daily Cases')
    index = searchSoup('Daily New Cases', data)
    #print(data[1])
    substring = re.search(patternData, data[index].string)
    #print(substring[1])
    dailyCases = pd.Series(
        substring[1].replace('null', '0').split(','),
        name='Daily Cases').astype(int)
    #print(dailyCases)

    #print('*****************Daily Death')
    index = searchSoup('Daily Deaths', data)
    #print(data[4])
    substring = re.search(patternData, data[index].string)
    #print(substring[1])
    dailyDeath = pd.Series(
        substring[1].replace('null', '0').split(','),
        name='Daily Death').astype(int)
    #print(dailyDeath)

    #print('*****************Total Death')
    index = searchSoup('Total Deaths', data)
    #print(data[3])
    substring = re.search(patternData, data[index].string)
    #print(substring[1])
    totalDeath = pd.Series(
        substring[1].replace('null', '0').split(','),
        name='Total Death').astype(int)
    #print(totalDeath)

    print('*****************DataFrame Construction')
    df = pd.concat([dayDate, dailyCases, totalCases, dailyDeath, totalDeath],
                   axis=1)
    print(df)

    print('*****************CSV')
    countryName = re.search(re.compile('country/(.*?)/'), url)
    #print(countryName[1])
    df.to_csv('/content/drive/My Drive/Covid19_Files/Covid19_' + countryName[1] + '.csv')


#*******************prg start**************************

getCountriesList()

for country in countries:
  print('\n*************************',country,'*************************')
  url = 'https://www.worldometers.info/coronavirus/' + country
  #print(url)
  getCountryStatics(url)
